In [ ]:
from IPython.display import HTML

#HTML('''<script>
#code_show=true; 
#function code_toggle() {
# if (code_show){
# $('div.input').hide();
# } else {
# $('div.input').show();
# }
# code_show = !code_show
#} 
#$( document ).ready(code_toggle);
#</script>
#<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [1]:
import pymongo
import numpy
#import colorlover as cl

client = pymongo.MongoClient("localhost", 27017)

db = client.test # Use DB name 'test'
collection = db.TRE_Performance_Mask

In [9]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

cursor = collection.find()
def f_channel_type(channel_type, rx_channel_type):
    if channel_type == 0:
        cursor = collection.find({'HFF_Channel_Type':'TX'}, {'TRE_Performance_Mask':'true', '_id':'False'}) # Find all TX records and return only array
    elif channel_type == 1:
        if rx_channel_type == 0:
            cursor = collection.find({'HFF_Channel_Type':'RX', 'HFF_RX_Channel_type':'HFF_A_CHANNEL'}, {'TRE_Performance_Mask':'true', '_id':'False'})
        elif rx_channel_type == 1:
            cursor = collection.find({'HFF_Channel_Type':'RX', 'HFF_RX_Channel_type':'HFF_B_CHANNEL'}, {'TRE_Performance_Mask':'true', '_id':'False'})
        else:
            print('ERROR: Unknown RX channel type')
    else:
        print('ERROR: Unknown channel type')
    
interact(f_channel_type, channel_type={'TX':0, 'RX':1}, rx_channel_type={'A':0, 'B':1});

In [11]:
#cursor = collection.find()
#cursor = collection.find({'Temperature_Vec_1.Mean':30}, {'Temperature_Vec_1.Mean': 1})
#cursor = collection.find({'Temperature_Vec_1.Mean':30}, {'Temperature_Vec_1.$.Mean':30})
#cursor = collection.find({}, {'Temperature_Vec_1.Mean':'true', '_id':'false'})
#cursor = collection.find({}, {'Temperature_Vec_1':'true'})

# Find number of records
z = 0
for record in collection.find():
    z += 1
    print(z)

# Matrix with all TRE values. x=col, y=row, z=records
values = numpy.array([[[0 for x in range(40)] for y in range(48)] for z in range(z-1)])
# Index using [z][y][x] from now on. First index is 0
#print(values)

# For all TRE_Performance_Mask records
record_index = 0
for record in cursor:
    #print(record)
    #print(record_index)
    values[record_index] = record['TRE_Performance_Mask']
    record_index += 1

#print(values)
#print(values[0])

1
2


## Create interactive plot here

In [ ]:
####### Plotting using plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
init_notebook_mode()
    
from ipywidgets import interactive, interact
import ipywidgets as widgets
from IPython.display import display

row_label = []
for i in range(48):
    row_label.append(str(i+1))
col_label = []
for i in range(40):
    col_label.append(str(i+1))
#bupu = cl.scales['9']['seq']['BuPu']
#bupu500 = cl.interp( bupu, 500 ) # Map color scale to 500 bins
#colorscale = [[0, '#3D9970'], [150, '#001f3f']]
    
def data_selector(z):
    selected_values = values[z]
    #print(selected_values.shape)
    
    # Transpose x and y in array
    #values_trans = numpy.array(selected_values).transpose()

    fig = FF.create_annotated_heatmap(selected_values, y=row_label, x=col_label, colorscale='Jet') # 'Viridis'
    fig['layout'].update({
        'title':'HFF_MRM_Temperature_Status',
        'autosize':False,
        'width':'800',
        'height':'1000',
        #'margin':go.Margin(l=50, r=50, b=100, t=100, pad=4)
        #'paper_bgcolor':'#7f7f7f',
        #'plot_bgcolor':'#c7c7c7'
    })
    iplot(fig)
    
    #print('Selected index=' + str(z))
    #print(selected_values)
    
#data_selector(0)

In [ ]:
interact(data_selector, z=widgets.IntSlider(min=0, max=len(values)-1, step=1, value=0));